# Analyse E-commerce Product Reviews using NLP 

## **Import Packages**

In [ ]:
# Import the generic libraries
import pandas as pd
import numpy as np
from collections import defaultdict
from collections import Counter
import csv

# Importing nltk libraries
import nltk
nltk.download('all')
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
lemmatizer = nltk.stem.WordNetLemmatizer()
wordnet_lemmatizer = WordNetLemmatizer()


# Visualization libraries
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from PIL import Image # for world cloud image

# Spacy for preprocessing
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load('en_core_web_sm')

#Spell checker
!pip install pyspellchecker
from spellchecker import SpellChecker

# Modelling
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

# xgboost
import xgboost as xgb
#
import tensorflow as tf
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from tensorflow.keras.optimizers import Adam


# To change date to datetime
from datetime import datetime
import time
import re 

from collections import Counter
import string
import scipy.sparse

# Textblob
from textblob import TextBlob

# Gensim libraries
!pip install pyLDAvis
from gensim import corpora, models, similarities, matutils
from gensim.models.ldamulticore import LdaMulticore
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
from gensim.models import CoherenceModel


# To show all the columns
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_colwidth', 300)

# to pickle dataframe
import pickle

# Avoid warnings
import warnings
warnings.filterwarnings("ignore")


# Enable logging for gensim - optional but important
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

from IPython.display import clear_output
clear_output()

## **Load the Dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Datasets/GrammarandProductReviews.csv")

In [ ]:
df.head(3)

# **Text Pre-processing**

In [ ]:
print("Shape :", df.shape)

In [ ]:
print("Columns :")
print(df.columns)

### **Rename the Columns**

In [ ]:
# Rename the column names
col_names = df.columns
new_col_names = [i.replace(".","_") for i in col_names]
df.columns = new_col_names

In [ ]:
df.columns

In [ ]:
print("Datatypes :\n",df.dtypes)

In [ ]:
print("Info :")
print(df.info())

In [ ]:
print("Missing Value Count :")
print(df.isnull().sum())

### **Percentage of missing values per column**

In [ ]:
print("Percentage of missing values :")
print(df.isna().mean().round(4) * 100)

In [ ]:
#Visualization of Missing Values

# Plot it
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.heatmap(data=df.isnull())
plt.xticks(rotation=90, fontsize=10)
plt.yticks(rotation=0, fontsize=10)
plt.show()



In [ ]:
print("Total Missing Value Count : ", df.isnull().sum().values.sum())

In [ ]:
# Drop the columns with less than 20% of values
missing_val_threshold = len(df) * .2
df.dropna(thresh = missing_val_threshold, axis = 1, inplace = True)

In [ ]:
df.columns

In [ ]:
# Shape of Dataset
df.shape

In [ ]:
# Drop the rows where "reviews.text" or "reviews.date" feature has Null values
df.dropna(subset=['reviews_text','reviews_date'], inplace=True)

In [ ]:
# Shape of Dataset
df.shape

In [ ]:
print("Percentage of missing values :")
print(df.isna().mean().round(4) * 100)

### **Combine Review Text and Title into one**

In [ ]:
# Joining Review Text and Title 
df['Review'] = df['reviews_title'].map(str) + " " + df['reviews_text'] 

### **Lowercasing**

In [ ]:
# Lowercasing the reviews and title column
df['Review'] = df['Review'].apply(lambda x : x.lower())

In [ ]:
df['Review'][0]

### **Remove Punctuation**

In [ ]:
# Remove punctuation 
df['Review'] = df['Review'].str.replace('[^\w\s]','')

In [ ]:
df['Review'][0]

### **Remove Stopwords**

In [ ]:
# Remove Stopwords
stop = stopwords.words('english')
df['Review'] = df['Review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [ ]:
df['Review'][0]

### **Lemmatization**

In [ ]:
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

In [ ]:
def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

In [ ]:
df['Review']=df['Review'].apply(lambda x: lemmatize_sentence(x))

In [ ]:
df['Review'][0]

### **Spelling Correction**

In [ ]:
df['Review']=df['Review'].apply(lambda x: re.sub('[^A-Za-z0-9]+', ' ', x))
df['Review'] = df['Review'].apply(lambda x: ''.join([i for i in x if not i.isdigit()]))
df['Review'] = df['Review'].apply(lambda x: x.strip())

In [ ]:
df['Review'].head(10)

In [ ]:
!pip install autocorrect
from autocorrect import Speller

spell = Speller(lang='en')

# def autospell(text):
#         spells = [spell(w) for w in (nltk.word_tokenize(text))]
#         return " ".join(spells)

In [ ]:
df['Review']=df['Review'].apply(lambda x: ' '.join(spell(word) for word in nltk.word_tokenize(x)))

In [ ]:
#df.to_csv("/content/drive/My Drive/Updated_GrammarandProductReviews.csv")

In [ ]:
df = pd.read_csv("/content/drive/My Drive/Updated_GrammarandProductReviews.csv")

### **Normalization**
https://sentic.net/microtext-normalization.pdf

### **Standarization**

### **Noise Removal**

In [ ]:
def scrub_words(text):
    """Basic cleaning of texts."""
    
    # remove html markup
    text=re.sub("(<.*?>)","",text)
    
    #remove non-ascii and digits
    text=re.sub("(\\W|\\d)"," ",text)
    
    #remove whitespace
    text=text.strip()
    return text

In [ ]:
df['Review']=df['Review'].apply(lambda x: scrub_words(x))

In [ ]:
df['Review'][0]

### **Word Count**

In [ ]:
df['Review_WC'] = df['Review'].apply(lambda x: len(str(x).split(" ")))
df[['Review_WC','Review']].head(3)

In [ ]:
# Density Plot and Histogram of all Word Count
sns.distplot(df['Review_WC'], hist=True, kde=True, 
             bins=int(180/5), color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})

### **Character Count**

In [ ]:
df['Review_CC'] = df['Review'].str.len() ## this also includes spaces
df[['Review_CC','Review']].head(3)

In [ ]:
# Density Plot and Histogram of all Character Count
sns.distplot(df['Review_CC'], hist=True, kde=True, 
             bins=int(180/5), color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})

### **Average Word Length**

In [ ]:
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))

df['Review_AWL'] = df["Review"].apply(lambda x: avg_word(x))
df[['Review_AWL','Review']].head(3)

In [ ]:
# Density Plot and Histogram of Average Word Length
sns.distplot(df['Review_AWL'], hist=True, kde=True, 
             bins=int(180/5), color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})

### **Top 30 Common Words**

In [ ]:
# function to plot most frequent terms
def freq_words(x, terms = 30):
  all_words = ' '.join([text for text in x])
  all_words = all_words.split()

  fdist = FreqDist(all_words)
  words_df = pd.DataFrame({'word':list(fdist.keys()), 'count':list(fdist.values())})

  # selecting top 20 most frequent words
  d = words_df.nlargest(columns="count", n = terms) 
  plt.figure(figsize=(20,10))
  ax = sns.barplot(data=d, x= "count", y = "word")
  ax.set(ylabel = 'Word')
  plt.show()

In [ ]:
freq_words(df['Review'])

### **Top 30 Rare Words**

In [ ]:
# function to plot least frequent terms
def freq_words(x, terms = 30):
  all_words = ' '.join([text for text in x])
  all_words = all_words.split()

  fdist = FreqDist(all_words)
  words_df = pd.DataFrame({'word':list(fdist.keys()), 'count':list(fdist.values())})

  # selecting top 20 most frequent words
  d = words_df.nsmallest(columns="count", n = terms) 
  plt.figure(figsize=(20,10))
  ax = sns.barplot(data=d, x= "count", y = "word")
  ax.set(ylabel = 'Word')
  plt.show()

In [ ]:
freq_words(df['Review'])

### **Term Frequency**

In [ ]:
df_tf = (df['Review'][0:1]).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
df_tf.columns = ['words','tf']
df_tf

### **IDF**

In [ ]:
for i,word in enumerate(df_tf['words']):
  df_tf.loc[i, 'idf'] = np.log(df_tf.shape[0]/(len(df[df['Review'].str.contains(word)])))


In [ ]:
df_tf

### **TF-IDF**

In [ ]:
df_tf['tfidf'] = df_tf['tf'] * df_tf['idf']
df_tf

### **Transforming Reviews Date to Python DateTime Format**

In [ ]:
# Convert the Rating Date column in datetime format
df['reviews_date'] = df['reviews_date'].str.replace(".000Z","")
df['reviews_date'] = df['reviews_date'].str.replace("Z","")

In [ ]:
date = []
for i,j in enumerate(df['reviews_date']): 
  try :
    date.append(datetime.strptime(j, "%Y-%m-%dT%H:%M:%S"))
  except:
    print(i)
    j = df['reviews_date'][0]
    date.append(datetime.strptime(j, "%Y-%m-%dT%H:%M:%S"))

  
df['reviews_date'] = date

# **Exploratory Data Analysis**

### **Unique Values**

In [ ]:
for i in ['brand', 'categories','manufacturer','name','reviews_id','reviews_rating']:
  print("No. of unique values in %s is : %s" %(i, df[i].nunique()))

### **Distribution of top 25 reviewed brands**

In [ ]:
df['brand'] = df['brand'].replace("L'oreal Paris","L'Oreal Paris")
df['brand'] = df['brand'].replace("Sony","Sony Pictures")

In [ ]:
df['brand'].value_counts()[0:25].sort_values().plot(kind = 'barh')

### **Distribution of reviewed categories**

In [ ]:
df["product_cat"] = df["categories"].apply(lambda x: x.split(",")[0])

df['product_cat'] = df['product_cat'].replace("Movies","Movies & TV Shows")
df['product_cat'] = df['product_cat'].replace("Food","Food & Beverage")
df['product_cat'] = df['product_cat'].replace("Household Chemicals","Household Essentials")
df['product_cat'] = df['product_cat'].replace("Music on CD or Vinyl","Musical Instruments & Karaoke")
df['product_cat'] = df['product_cat'].replace("Pro Audio","Musical Instruments & Karaoke")
df['product_cat'] = df['product_cat'].replace("Baby","Kids")
df['product_cat'] = df['product_cat'].replace("Kids' Rooms","Kids")
df['product_cat'] = df['product_cat'].replace("Brand Shop","Household Essentials")
df['product_cat'] = df['product_cat'].replace("Featured Brands","Household Essentials")
df['product_cat'] = df['product_cat'].replace("Home Improvement","Household Essentials")
df['product_cat'] = df['product_cat'].replace("Furniture","Household Essentials")
df['product_cat'] = df['product_cat'].replace("Video Games","Sports & Outdoors")
df['product_cat'] = df['product_cat'].replace("Home","Household Essentials")
df['product_cat'] = df['product_cat'].replace("Mobile","Electronics")
df['product_cat'] = df['product_cat'].replace("Photography","Electronics")
df['product_cat'] = df['product_cat'].replace("Patio & Garden","Accessories")
df['product_cat'] = df['product_cat'].replace("Gift Finder","Accessories")
df['product_cat'] = df['product_cat'].replace("Auto & Tires","Accessories")
df['product_cat'] = df['product_cat'].replace("Kitchen & Dining","Food & Beverage")


In [ ]:
df['product_cat'].value_counts()[0:25].sort_values().plot(kind = 'barh')

### **Distribution of top 25 reviewed manufactures**

In [ ]:
df['manufacturer'] = df['manufacturer'].replace("L'oreal Paris","L'Oreal Paris")
df['manufacturer'] = df['manufacturer'].replace("SONY CORP","Sony Pictures")
df['manufacturer'].value_counts()[0:25].sort_values().plot(kind = 'barh')

### **Distribution of Review Ratings**

In [ ]:
# Density Plot and Histogram of Reviews Ratings
sns.countplot(df['reviews_rating'])

### **Year Wise Ratings**

In [ ]:
df['year'], df['day'], df['month'] = df['reviews_date'].dt.year, df['reviews_date'].dt.day, df['reviews_date'].dt.month

In [ ]:
df['hour'], df['minute'], df['second'] = df['reviews_date'].dt.hour, df['reviews_date'].dt.minute, df['reviews_date'].dt.second

In [ ]:
sns.countplot(y=df['year'])

In [ ]:
sns.countplot(y=df['year'], hue=df['reviews_rating'])

### **Hour Wise Ratings**

In [ ]:
sns.countplot(y=df['hour'])

In [ ]:
sns.countplot(y=df['hour'], hue=df['reviews_rating'])

### **Category vs Ratings**

In [ ]:
df['product_cat'].nunique()

In [ ]:
sns.countplot(y=df['product_cat'], hue=df['reviews_rating'])

In [ ]:
sns.countplot(y=df['reviews_rating'], hue=df['product_cat'])

### **Brands vs Ratings**

In [ ]:
sns.countplot(y=df['brand'], hue=df['reviews_rating'], order=df['brand'].value_counts().iloc[:20].index)

### **Word Clouds**

In [ ]:
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)

def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color='white',
        stopwords=stopwords,
        max_words=300,
        max_font_size=40, 
        scale=3,
        random_state=1    ).generate(str(data))

    fig = plt.figure(1, figsize=(15, 15))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()

show_wordcloud(df['Review'])

In [ ]:
show_wordcloud(df['reviews_title'])
# Great = 10938, great = 3133
# Disappointed = 156, disappointed = 75, Disappointing = 50, disappointing = 25, dissapoitned = 

In [ ]:
# try to tokenize to individual word (uni-gram) - reviews.text
split_title = []
listCounts = []
split_title = [x.split(" ") for x in df['Review'].astype(str)]
big_list = []
for x in split_title:
    big_list.extend(x)

listCounts = pd.Series(big_list).value_counts()

wordcloud = WordCloud(background_color='white', max_words=400, max_font_size=40, scale=30,
        random_state=1).generate((listCounts[listCounts > 2]).to_string())
plt.figure(figsize=(15, 15))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
len(big_list)   # reviews_text


### **Genuine Reviews**

In [ ]:
# on the reviews.didPurchase column, replace 38,886 null fields with "Null"
df['reviews_didPurchase'].fillna('Null', inplace=True)


In [ ]:
plt.figure(figsize=(10,8))
ax = sns.countplot(df['reviews_didPurchase'])
ax.set_xlabel(xlabel="Shoppers did purchase the product", fontsize=17)
ax.set_ylabel(ylabel='Count of Reviews', fontsize=17)
ax.axes.set_title('Number of Genuine Reviews', fontsize=17)
ax.tick_params(labelsize=13)

In [ ]:
df['reviews_didPurchase'].value_counts()


In [ ]:
# shoppers who did purchased the product and provided the review = 5%
print("Percentage of genuine reviews :",368200/(38785 + 28474))

### **Correlation Map**

In [ ]:
# not much info in the correlation map
sns.set(font_scale=1.4)
plt.figure(figsize = (10,5))
sns.heatmap(df.corr(),cmap='coolwarm',annot=True,linewidths=.5)

### **Most Bought Product**

In [ ]:
# "The Foodsaver174 10 Cup Fresh Container - Fac10-000" is purchased almost 500 times
df_genuine = df[df['reviews_didPurchase'] == True]
df_genuine['name'].value_counts()

In [ ]:
df_genuine['name'].value_counts()[0:10].plot(kind ='barh', figsize=[10,6], fontsize=20).invert_yaxis()

### **Most purchased Product - 5 Rating**

In [ ]:
df_mbp = df_genuine[df_genuine['name'] == 'The Foodsaver174 10 Cup Fresh Container - Fac10-000']
df_mbp = df_mbp[df_mbp['reviews_rating']==5]
# keep relevant columns only
df_mbp = df_mbp[[ 'reviews_rating', 'Review']]
df_mbp.head(3)

### **Most purchased Product - 1 Rating**

In [ ]:
# filter most purchased product with 1 star rating
df_lvp = df_genuine[df_genuine['name'] == 'The Foodsaver174 10 Cup Fresh Container - Fac10-000']
df_lvp = df_lvp[df_lvp['reviews_rating']==1]
# keep relevant columns only
df_lvp = df_lvp[[ 'reviews_rating', 'Review']]
df_lvp.head(3)

# **Training Data Preparation**

### **Defining features and target variables**

In [ ]:
x=df['Review']
y=df['reviews_rating']


### **Using the n-gram tfidf vectorizer**

In [ ]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 3) )  

word_vectorizer.fit(x)
train_word_features = word_vectorizer.transform(x)

In [ ]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6),
    max_features=50000)

char_vectorizer.fit(x)
train_char_features = char_vectorizer.transform(x)

train_features = hstack([train_char_features, train_word_features])

### **Splitting the dataset into train and test**

In [ ]:
seed = 50 

X_train, X_test, y_train, y_test = train_test_split(train_features, y, test_size=0.3, random_state=seed)
print('X_train', X_train.shape)
print('y_train', y_train.shape)
print('X_test', X_test.shape)
print('y_test', y_test.shape)

# **ML Model**

### **Random Forest Classifier**

In [ ]:
time1 = time.time()

classifier = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=seed, n_jobs=-1)
classifier.fit(X_train,y_train)

preds1 = classifier.predict(X_test)

time_taken = time.time() - time1
print('Time Taken: {:.2f} seconds'.format(time_taken))

In [ ]:
print("Random Forest Model accuracy", accuracy_score(preds1, y_test))
print(classification_report(preds1, y_test))
print(confusion_matrix(preds1, y_test))

### **Logistic Regression**

In [ ]:
time1 = time.time()

logit = LogisticRegression(C=1, multi_class='ovr')
logit.fit(X_train,y_train)
preds3 = logit.predict(X_test)

time_taken = time.time() - time1
print('Time Taken: {:.2f} seconds'.format(time_taken))

In [ ]:
print("Logistic Regression accuracy", accuracy_score(preds3, y_test))
print(classification_report(preds3, y_test))
print(confusion_matrix(preds3, y_test))

# **Deep Learning Model**

### **Label the Ratings** 



In [ ]:
# To classify ratings<4 as sentiment, i.e. replace ratings less than 4 as not happy
# label 1= Happy
# label 2= Unhappy

df['sentiment'] = df['reviews_rating']<4
train_text, test_text, train_y, test_y = train_test_split(df['Review'],df['sentiment'],test_size = 0.2)

In [ ]:
MAX_NB_WORDS = 20000

# get the raw text data
texts_train = train_text.astype(str)
texts_test = test_text.astype(str)

# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS, char_level=False)
tokenizer.fit_on_texts(texts_train)
sequences = tokenizer.texts_to_sequences(texts_train)
sequences_test = tokenizer.texts_to_sequences(texts_test)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [ ]:
MAX_SEQUENCE_LENGTH = 200
#pad sequences are used to bring all sentences to same size.
# pad sequences with 0s
x_train = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
x_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', x_train.shape)
print('Shape of data test tensor:', x_test.shape)

In [ ]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2,input_shape=(1,)))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

[Optimizers](https://towardsdatascience.com/optimizers-for-training-neural-network-59450d71caf6)

In [ ]:
model.fit(x_train, train_y,
          batch_size=128,
          epochs=1,
          validation_data=(x_test, test_y))

In [ ]:
#model.evaluate(X_test,y_test)